# State change propagation

In [18]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [19]:
class SubModel(bu.Model):
    name = 'submodel'
    length = bu.Float(2, GEO=True)

    @tr.observe('state_changed')
    def submodel_changed(self, event):
        print('submodel changed')

    ipw_view = bu.View(
        bu.Item('length')
    )

In [20]:
sm = SubModel(state_change_debug=True)

In [21]:
sm.length = 3

state_changed <__main__.SubModel object at 0x7f4630d47790> TraitChangeEvent(object=<__main__.SubModel object at 0x7f4630d47790>, name='length', old=2, new=3.0)
submodel changed


The change of a model parameter is observed and notified
by an event `sm.state_changed`. Let us observe this event
using a function


After executing the above cell, change the length in the
above model window to see the spy in action.

How did it happen? The `length` parameter of the `SubModel`
contains the metadate `GEO=True`. This indicates the type of
model input data. Except of `GEO` there are also the following
types of input data:

 - `BC` - boundary condition
 - `MAT` - material parameter
 - `DSC` - discretization
 - `CS` - cross section
 - `ALG` - algorithm parameter
 - `GEO` - global geometry

Change of any parameter denoted by one of the above tags
is observed and propagated along the model tree.

Let us define a more complex model

In [22]:
class InterimModel(bu.Model):
    sm = bu.Instance(SubModel,(), GEO=True)
    stiffness = bu.Float(400, GEO=True)

    tree = ['sm']
    @tr.observe('state_changed')
    def submodel_changed(self, event):
        print('interim model changed')


In [23]:
im = InterimModel(state_change_debug=True)

In [24]:
im.sm.length = 7

submodel changed
state_changed <__main__.InterimModel object at 0x7f4631b0e610> TraitChangeEvent(object=<__main__.SubModel object at 0x7f463156b330>, name='state_changed', old=<undefined>, new=True)
interim model changed


In [25]:
class TopLevelModel(bu.Model):
    im = bu.Instance(InterimModel,(), DSC=True)
    time = bu.Float(400, ALG=True)

    tree = ['im']
    @tr.observe('state_changed')
    def toplevel_changed(self, event):
        print('toplevel model changed')



In [26]:
tl = TopLevelModel(state_change_debug=True)
tl.im.state_change_debug = True
tl.im.sm.state_change_debug = True

In [27]:
tl.im.sm.length = 10

state_changed <__main__.SubModel object at 0x7f4631110d60> TraitChangeEvent(object=<__main__.SubModel object at 0x7f4631110d60>, name='length', old=2, new=10.0)
submodel changed
state_changed <__main__.InterimModel object at 0x7f4630e034c0> TraitChangeEvent(object=<__main__.SubModel object at 0x7f4631110d60>, name='state_changed', old=<undefined>, new=True)
interim model changed
state_changed <__main__.TopLevelModel object at 0x7f4630e039c0> TraitChangeEvent(object=<__main__.InterimModel object at 0x7f4630e034c0>, name='state_changed', old=<undefined>, new=True)
toplevel model changed


# Test a list

In [28]:
class CrossSectionLayout(bu.ModelList):
    name = 'Cross Section Layout'

    items = bu.List(SubModel, [])

    def add_layer(self, rl):
        self.items.append(rl)

In [29]:
csl = CrossSectionLayout(state_change_debug=True)

In [30]:
csl.add_layer(SubModel(state_change_debug=True))

LIST STATE CHANGED TraitChangeEvent(object=<__main__.CrossSectionLayout object at 0x7f4630c4ab10>, name='items_items', old=<undefined>, new=TraitListEvent(index=0, removed=[], added=[<__main__.SubModel object at 0x7f4630c485e0>]))


In [31]:
csl.items[0].name

'submodel'